# Deep Neural Network for MNIST Classification
## Assignment 1
### Programming Questions

In [27]:
# Import neccesary dependencies
from pytorch_lightning import LightningModule, Trainer
from torch import nn
from torch.nn.functional import cross_entropy
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
from torchmetrics import Accuracy
from torch.utils.data import DataLoader, random_split
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning.callbacks import EarlyStopping
import torchvision
import torch

## Load Dataset

In [28]:

train_set = torchvision.datasets.MNIST("./files/", train=True, download=True,
                                       transform=torchvision.transforms.Compose([
                                          torchvision.transforms.ToTensor(),
                                          torchvision.transforms.Lambda(lambda x: torch.reshape(x, (784,)))
                                       ]))

test_set = torchvision.datasets.MNIST("./files/", train=False, download=True,
                                      transform=torchvision.transforms.Compose([
                                          torchvision.transforms.ToTensor(),
                                          torchvision.transforms.Lambda(lambda x: torch.reshape(x, (784,)))
                                      ]))

train_set, val_set = random_split(train_set, [54000, 6000])

# Print the sizes of the training, validation, and test sets
print(f"Training set size: {len(train_set)}")  # Print the size of the training set
print(f"Validation set size: {len(val_set)}")  # Print the size of the validation set
print(f"Test set size: {len(test_set)}")       # Print the size of the test set

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=8)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=8)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=8)

Training set size: 54000
Validation set size: 6000
Test set size: 10000


In [29]:
# Neural Network Architecture Definition

class MNISTClassificator(LightningModule):
    def __init__(self):
        super().__init__()
        
        self.input = nn.Sequential(
            nn.Linear(784, 784),
            nn.BatchNorm1d(784),
            nn.ReLU(inplace=True)
        )
        
        self.layer1 = nn.Sequential(
            nn.Linear(784, 800),
            nn.BatchNorm1d(800),
            nn.ReLU(inplace=True)
        )
        
        self.layer2 = nn.Sequential(
            nn.Linear(800, 400),
            nn.BatchNorm1d(400),
            nn.ReLU(inplace=True)
        )
        
        self.layer3 = nn.Sequential(
            nn.Linear(400, 150),
            nn.BatchNorm1d(150),
            nn.ReLU(inplace=True)
        )
        
        self.layer4 = nn.Sequential(
            nn.Linear(150, 50),
            nn.BatchNorm1d(50),
            nn.ReLU(inplace=True)
        )
        
        self.output = nn.Sequential(
            nn.Linear(50, 10),
            nn.BatchNorm1d(10),
            nn.Softmax(dim=1)
        )
        
        self.train_acc = Accuracy(task="multiclass", num_classes=10)
        self.val_acc = Accuracy(task="multiclass", num_classes=10)
        self.test_acc = Accuracy(task="multiclass", num_classes=10)
        
    def forward(self, x):
        x = self.input(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        y = self.output(x)
        return y
    
# Optimizer
    def configure_optimizers(self):
        optimizer = Adam(self.parameters(), lr=0.02)
        lr_scheduler = StepLR(optimizer=optimizer, step_size=2)
        
# Training, Validation and Test Step
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = cross_entropy(y_hat, y)
        self.log("train_loss", loss, on_step=True)
        self.train_acc(y_hat, y)
        self.log("train_acc", self.train_acc, on_epoch=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        val_loss = cross_entropy(y_hat, y)
        self.log("val_loss", val_loss, on_step=True)
        self.val_acc(y_hat, y)
        self.log("val_acc", self.val_acc, on_epoch=True)
        
    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        test_loss = cross_entropy(y_hat, y)
        self.log("test_los", test_loss, on_step=True)
        self.test_acc(y_hat, y)
        self.log("test_acc", self.test_acc, on_epoch=True)

In [30]:
# Training and Evaluation
batch_size = 500
num_epochs = 50

model = MNISTClassificator()

tb_logger = pl_loggers.TensorBoardLogger(save_dir="logs/")

trainer = Trainer(devices=1, accelerator="auto", max_epochs=num_epochs, default_root_dir="/hands_on", logger=tb_logger,
                     callbacks=EarlyStopping(monitor="val_loss", min_delta=0.01, patience=10, verbose=True))

trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)
trainer.test(ckpt_path="best", dataloaders=test_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type               | Params
-------------------------------------------------
0 | input     | Sequential         | 617 K 
1 | layer1    | Sequential         | 629 K 
2 | layer2    | Sequential         | 321 K 
3 | layer3    | Sequential         | 60.5 K
4 | layer4    | Sequential         | 7.7 K 
5 | output    | Sequential         | 530   
6 | train_acc | MulticlassAccuracy | 0     
7 | val_acc   | MulticlassAccuracy | 0     
8 | test_acc  | MulticlassAccuracy | 0     
-------------------------------------------------
1.6 M     Trainable params
0         Non-trainable params
1.6 M     Total params
6.546     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Metric val_loss improved. New best score: 2.305


...

Validation: |          | 0/? [00:00<?, ?it/s]

Monitored metric val_loss did not improve in the last 10 records. Best score: 2.305. Signaling Trainer to stop.
Restoring states from the checkpoint path at logs/lightning_logs/version_2/checkpoints/epoch=10-step=1188.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at logs/lightning_logs/version_2/checkpoints/epoch=10-step=1188.ckpt


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.09719999879598618    │
│      test_los_epoch       │    2.3068459033966064     │
└───────────────────────────┴───────────────────────────┘

[{'test_los_epoch': 2.3068459033966064, 'test_acc': 0.09719999879598618}]